## 📊 分析の流れ
<details>
<summary><b>目次</b></summary>

1. **データ読み込み**
1. **欠損値処理**
    - 欠損列処理  
    - 欠損値処理  
    - 異常値処理  
1. **集計**
1. **データ出力**
</details>

In [ ]:
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


## 📥 データの読み込み

In [ ]:
# 設定読み込み
with open("setting.yaml", "r", encoding="utf-8") as f:
    setting = yaml.safe_load(f)
base_path = setting["data_path"]
years = setting["years"]
files = setting["files"]
files_reference = setting["files_reference"]
output_path = setting["output"]["base_path"]
output_files = setting["output"]["files"]
# 出力ファイルのフォルダ作成
os.makedirs(output_path, exist_ok=True)

## 🔍 欠損列処理
4つファイルの列が各年度で同じかを目視で確認しておきます。
問題がある場合は処理します。
<details><summary><b>結果</b></summary>
すべてのファイルで同じ列が登録されています。問題ありません。

- fy-balance-sheet.csv
- fy-cash-flow-statement.csv
- fy-profit-and-loss.csv
- fy-stock-dividend.csv
</details>


In [ ]:
for filename in files:
    merged = []
    for year in years:
        file_path = os.path.join(base_path, str(year), filename)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path,header=1)
            print("年：",year," ","ファイル名:",filename," ","列",df.columns.to_list())

## 🧹 欠損値処理
ここでは、後の分析・スクリーニングの処理を適切に設計するため、欠損値の傾向を確認します。  
⚠️ **注意※** csvファイルには欠損値に"-"が入っているため、NaNに置き換えて読み込みます。
<details><summary><b>分析内容</b></summary>

- 列ごとの欠損値の数・割合を確認する
- 年ごとの欠損率の傾向を確認
- 全企業の欠損率分布を見る
<br>
</details>
<details><summary><b>結果と考察</b></summary>

- fy-balance-sheet.csv
- fy-cash-flow-statement.csv
- fy-profit-and-loss.csv
- fy-stock-dividend.csv
</details>


### ４つのファイル結合

In [ ]:
all_df = {}
missing_year_df = {}
for filename in files:
    if filename not in missing_year_df:
        all_df[filename] = pd.DataFrame()
        missing_year_df[filename] = pd.DataFrame()
    for year in years:
        file_path = os.path.join(base_path, str(year), filename)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path,header=1,na_values="-")
            all_df[filename] = pd.concat([all_df[filename], df], ignore_index=True) # 全データ結合
            missing_rate = df.isnull().mean().to_frame(name="missing_rate")
            missing_rate["year"] = year
            missing_rate = missing_rate.reset_index().rename(columns={"index": "column_name"})
            missing_year_df[filename] = pd.concat([missing_year_df[filename], missing_rate], ignore_index=True)
#display(all_df[files[0]].head(100))

### 列ごとの欠損値の割合を確認する

In [ ]:

missing_column_df = {}
for filename in files:
    if filename not in missing_column_df:
        missing_column_df[filename] = pd.DataFrame()
    missing_column_df[filename] = all_df[filename].isna().mean()
    fig = px.bar(missing_column_df[filename], x=missing_column_df[filename].index,
                    y=missing_column_df[filename].values)
    fig.show()

### 年ごとの欠損率の傾向を確認（折れ線グラフ）

In [ ]:
# 各カラムの欠損率推移を折れ線グラフで表示
for filename in files:
    fig = px.line(
        missing_year_df[filename],  # 最初のファイルのデータを使用
        x="year",
        y="missing_rate",
        color="column_name",   # 列ごとに色分け
        markers=True,
        title="各列の欠損率の推移"
    )
    fig.show()

### 年ごとの欠損率の傾向を確認（折れ線グラフ）

In [ ]:
# 年ごとの欠損率の傾向を確認（ヒートマップ）

for filename in missing_year_df:
    pivot_table = missing_year_df[filename].pivot(index="column_name", columns="year", values="missing_rate")
    fig = go.Figure(data=go.Heatmap(
        z=pivot_table.values,
        x=pivot_table.columns,
        y=pivot_table.index,
        colorscale="Viridis",
        colorbar=dict(title="Missing Rate")
    ))
    fig.update_layout(
        title=f"{filename} の欠損率ヒートマップ",
        xaxis_title="Year",
        yaxis_title="Column Name"
    )
    fig.show()

### 全企業の欠損率分布を見る（ヒストグラム）

In [ ]:
# コードごとに平均欠損率を計算する
missing_score = {}
for filename in files:
    # コードごとに欠損率を計算
    missing_score[filename] = pd.DataFrame()
    missing_score[filename] = (
        all_df[filename]
        .groupby("コード")
        .agg(lambda x: x.isnull().mean())  # 各列ごとの欠損率
        .mean(axis=1)                      # 列平均
        .reset_index(name="平均欠損率")
    )
    #pd.set_option('display.max_rows', None)
    #display(missing_score[filename].info())

In [25]:
# 分析
target_file = files[0]

# ヒストグラム表示
fig = px.histogram(
    missing_score[target_file],  # 最初のファイルのデータを使用
    x="平均欠損率",
    nbins=50,
    title="全企業の欠損率分布"+files[0],
    labels={"平均欠損率": "平均欠損率"}
)
#fig.show()
# 特定範囲の欠損率を持つコードを抽出
threshold_low = 0.215
threshold_high= 0.225
filtered_codes = missing_score[target_file][
    (missing_score[target_file]["平均欠損率"] >= threshold_low) &
    (missing_score[target_file]["平均欠損率"] < threshold_high)
]
#display(filtered_codes)
# 列別欠損率
df_all = all_df[target_file] 
df_by_code = df_all.set_index("コード") # 企業コードを index にする
#print(df_by_code)
codes = filtered_codes["コード"].tolist() # 企業コードをリスト化
subset = df_by_code.loc[df_by_code.index.isin(codes)] # 企業コードのデータ抽出
# print(subset)
col_missing = subset.isnull().mean().sort_values(ascending=False) # 列別欠損率
display(col_missing)

#年次別の欠損率推移確認
df = all_df[target_file] 
year_all = df.groupby('年度')[['短期借入金','長期借入金']].apply(lambda d: d.isnull().mean())
year_all = year_all.reset_index()
print("全体 年次欠損率:\n", year_all)
# 欠損率の推移を可視化
fig = px.line(
    year_all,
    x="年度",
    y=["短期借入金", "長期借入金"],  # 複数列を同時に表示
    markers=True,
    title="各列の欠損率の推移"
)
fig.show()

短期借入金     0.979902
長期借入金     0.957614
利益剰余金     0.019196
BPS       0.018165
株主資本      0.017392
自己資本比率    0.002448
総資産       0.002061
純資産       0.000386
年度        0.000000
dtype: float64

全体 年次欠損率:
           年度     短期借入金     長期借入金
0    2010/01  0.461538  0.384615
1    2010/02  0.372881  0.361582
2    2010/03  0.343200  0.376662
3    2010/04  0.611111  0.416667
4    2010/05  0.500000  0.388889
..       ...       ...       ...
178  2024/11  0.400000  0.184615
179  2024/12  0.414480  0.343891
180  2025/01  0.492308  0.323077
181  2025/02  0.411215  0.266355
182  2025/03  0.336153  0.306768

[183 rows x 3 columns]


# OLD --- IGNORE ---

In [ ]:
# IR bankのデータから必要なデータを吸い上げる
import pandas as pd
import myLibStandard as myLib
myLib.set_DisplayOption()

# BSから必要なデータを取得
dfBalanceSheet = pd.DataFrame()
dl = ["fy-balance-sheet", "fy-cash-flow-statement", "fy-profit-and-loss", "fy-stock-dividend"]
for folder in range(2010, 2026):
    df = pd.read_csv(myLib.pathDataDomesticData + str(folder) + "/fy-balance-sheet.csv",\
    header=1, index_col=["コード", "年度"], dtype={"コード":"str"})
    dfBalanceSheet = pd.concat([dfBalanceSheet, df], axis=0, join="outer")
dfBalanceSheet = dfBalanceSheet.sort_index()
dfBalanceSheet = dfBalanceSheet.loc[~dfBalanceSheet.index.duplicated("last")]

# PLから必要なデータを取得
dfProftLoss = pd.DataFrame()
for folder in range(2010, 2026):
    df = pd.read_csv(myLib.pathDataDomesticData + str(folder) + "/" + "fy-profit-and-loss.csv",\
    header=1, index_col=["コード", "年度"], dtype={"コード":"str"})
    dfProftLoss = pd.concat([dfProftLoss, df], axis=0, join="outer")
dfProftLoss = dfProftLoss.sort_index()
dfProftLoss = dfProftLoss.loc[~dfProftLoss.index.duplicated("last")]

# SDから必要なデータを取得
dfStockDividend = pd.DataFrame()
for folder in range(2010, 2026):
    df = pd.read_csv(myLib.pathDataDomesticData + str(folder) + "/" + "fy-stock-dividend.csv",\
    header=1, index_col=["コード", "年度"], dtype={"コード":"str"})
    dfStockDividend = pd.concat([dfStockDividend, df], axis=0, join="outer")
dfStockDividend = dfStockDividend.loc[~dfStockDividend.index.duplicated("last")]
dfStockDividend = dfStockDividend.sort_index(level=["コード"])
#display(dfStockDividend)

dfAll = pd.concat([dfBalanceSheet, dfProftLoss], axis=1, join="outer")
dfAll = pd.concat([dfAll, dfStockDividend], axis=1, join="outer")
dfAll = dfAll.sort_index()
#display(dfAll.head(1000))
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv")


In [ ]:
# データがないセルを"-"からnullに変換する
import pandas as pd
import math
import myLibStandard as myLib
myLib.set_DisplayOption()

dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv",index_col=["コード", "年度"], dtype="str")
dfAll = dfAll.sort_index()
for column in dfAll:
    dfAll[column] = dfAll[column].replace("-", math.nan)
    dfAll[column] = dfAll[column].replace(0, math.nan)

dfAll.to_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv")

In [ ]:
# データが0の場合はNANにする（ゼロ割り算エラーを避けるため）
import pandas as pd
import math
import myLibStandard as myLib
myLib.set_DisplayOption()
dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv", dtype={"コード":"str"})

for i in dfAll.index:
    if dfAll.loc[i, "EPS"] == 0:
        dfAll.loc[i, "EPS"] = math.nan
    if dfAll.loc[i, "BPS"] == 0:
        dfAll.loc[i, "BPS"] = math.nan
dfAll = dfAll.set_index(["コード", "年度"])
#display(dfAll.head(1000))
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv")

##### 銘柄名や業種を取得

In [ ]:
# シンボルリスト取得、保存
from stocksymbol import StockSymbol
api_key = '63ed951b-b643-48b1-bfe7-90c7a4f96349'
ss = StockSymbol(api_key)
symbol_list_japan = ss.get_symbol_list(market="Japan") # "us" or "america" will also work
df = symbol_list_japan
df = pd.DataFrame(df)
df.to_csv(myLib.pathDataSummaryDomesticData + "symbol.csv")

In [ ]:
# Code一覧から銘柄の情報を取得し、登録する。ないものはデータから削除する
import pandas as pd
import myLibStandard as myLib
import warnings
#warnings.simplefilter('ignore')
myLib.set_DisplayOption()

dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "IR_Bank_Data.csv",index_col=["コード", "年度"], dtype={"コード":str})
dfCode = pd.read_csv(myLib.pathDataSummaryDomesticData + "CodeData.csv",usecols=["コード", "銘柄名", "市場・商品区分", "33業種区分"],\
index_col=["コード"], dtype=str)
dfAll["銘柄"] = None
dfAll["市場"] = None
dfAll["業種"] = None
dfAll = dfAll.sort_index()
dlCode = dfAll.groupby(level='コード').size().index.tolist()
dlDelCode = []
for code in dlCode:
    try:
        dfAll.loc[(code,), "銘柄"] = dfCode.loc[code, "銘柄名"]
        dfAll.loc[(code,), "市場"] = dfCode.loc[code, "市場・商品区分"]
        dfAll.loc[(code,), "業種"] = dfCode.loc[code, "33業種区分"]
    except KeyError:
        dlDelCode.append(code)

dfAll = dfAll.reset_index("年度")
for dl in dlDelCode:
    dfAll = dfAll.drop(dl,axis=0)
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DataPreprocessing.csv")

##### 銘柄のテーマ情報を取得し登録する

In [ ]:
# テーマ情報をCSVからとってきてテーマフラグを立てる
import pandas as pd
import myLibStandard as myLib
import warnings
myLib.set_DisplayOption()

dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DataPreprocessing.csv",index_col="コード", dtype={"コード":str})

def get_ThemeList(filname):
    df = pd.read_csv(myLib.pathDataDomesticData + filname, index_col="コード", dtype={"コード":str})
    return df

dlname = ["DOE採用銘柄", "円高メリット", "減配なし", "親子上場", "累進配当"]
for name in dlname:
    dfTheme = get_ThemeList(name + ".csv")
    for code in dfTheme.index:
        if code in dfAll.index:
            dfAll.loc[code, name] = 1

dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DataPreprocessing.csv")


## 分析値の計算、追加

##### データの登録（PER, PBR, MIX係数, 簡易理論株価, 安全域, 時価総額, 配当利回り, 資産価値を含めたPERを登録する）

In [ ]:
# PER, PBR, 簡易理論株価, 安全域, 配当利回り, MIX係数, 時価総額, を登録する (※長時間)
import pandas as pd
import math
import datetime
import calendar
from yahooquery import Ticker
import myLibStandard as myLib
import warnings
warnings.simplefilter('ignore', FutureWarning)
myLib.set_DisplayOption()

#dfSymbol = pd.read_csv(myLib.pathDataSummaryDomesticData + "symbol.csv", dtype={"symbol":str})
dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DataPreprocessing.csv",index_col=["コード", "年度"], dtype={"コード":"str"})
#dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv",index_col=["コード", "年度"], dtype={"コード":str})

dfAll["PER"] = math.nan
dfAll["PBR"] = math.nan
dfAll["簡易理論株価"] = math.nan
dfAll["安全域"] = math.nan
dfAll["配当利回り"] = math.nan
dfAll["MIX係数"] = math.nan
dfAll["時価総額"] = math.nan
dfAll["株価"] = math.nan
dfAll["営業利益率"] = math.nan
dfAll["経常利益率"] = math.nan
dfAll["純利益率"] = math.nan
dfAll["配当将来期待"] = round(dfAll["ROE"] - dfAll["純資産配当率"] ,1)

def get_StockValue(symbol):
    yqData = Ticker(symbol+".T")
    stockValue = yqData.history(start="2010-03-01")
    stockValue = stockValue.reset_index(level=["date",'symbol'])
    stockValue["date"] = stockValue["date"].apply(lambda x: pd.to_datetime(x).tz_localize(None))
    stockValue["date"] = pd.to_datetime(stockValue["date"])
    stockValue = stockValue.set_index("date")
    stockValue = stockValue["close"].resample("D").ffill()
    return stockValue
    #return pd.Series()

dfAll = dfAll.sort_index()
dlCode = dfAll.groupby(level='コード').size().index.tolist()
i=0
for code in dlCode:
    if (i>=0) and (i<5000):
        print(code)
        dsStock = get_StockValue(str(code))
        for date in dfAll.loc[(code,)].index:
            dtDate = datetime.datetime.strptime(date, '%Y/%m')
            dtDate = datetime.date(dtDate.year, dtDate.month, calendar.monthrange(dtDate.year, dtDate.month)[1])
            try:
                stockValue = dsStock.loc[dtDate.strftime("%Y-%m-%d")]
            except KeyError:
                stockValue = math.nan
            eps = float(dfAll.loc[(code, date), "EPS"])
            bps = float(dfAll.loc[(code, date), "BPS"])
            if eps < 0:
                dfAll.loc[(code, date), "PER"] = math.nan
            else:
                dfAll.loc[(code, date), "PER"] = round(stockValue / eps, 2)
            dfAll.loc[(code, date), "簡易理論株価"] = round(bps + eps * 10,2)
            dfAll.loc[(code, date), "PBR"] = round(stockValue / bps, 2)
            dfAll.loc[(code, date), "配当利回り"] = round(float(dfAll.loc[(code, date), "一株配当"] / stockValue *100),2)
            dfAll.loc[(code, date), "安全域"] = round(dfAll.loc[(code, date), "簡易理論株価"] / stockValue, 2)
            dfAll.loc[(code, date), "MIX係数"] = dfAll.loc[(code, date), "PER"] * dfAll.loc[(code, date), "PBR"]
            dfAll.loc[(code, date), "時価総額"] = dfAll.loc[(code, date), "PER"] * dfAll.loc[(code, date), "純利益"]
            dfAll.loc[(code, date), "株価"] = stockValue
            dfAll.loc[(code, date), "営業利益率"] = round(dfAll.loc[(code, date), "営業利益"] / dfAll.loc[(code, date), "売上高"] * 100 ,2)
            dfAll.loc[(code, date), "経常利益率"] = round(dfAll.loc[(code, date), "経常利益"] / dfAll.loc[(code, date), "売上高"] * 100 ,2)
            dfAll.loc[(code, date), "純利益率"] = round(dfAll.loc[(code, date), "純利益"] / dfAll.loc[(code, date), "売上高"] * 100 ,2)
    i+=1
    if i%10==0:
        print(code,i)
        dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv")
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv")

In [ ]:
# 資産価値を含めたPERを登録する (※長時間)
import pandas as pd
import math
import datetime
import calendar
from yahooquery import Ticker
import yfinance as yf
import myLibStandard as myLib
import warnings
#warnings.simplefilter('ignore', FutureWarning)
myLib.set_DisplayOption()

#dfSymbol = pd.read_csv(myLib.pathDataSummaryDomesticData + "symbol.csv", dtype={"symbol":"str"})
dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv",index_col=["コード", "年度"], dtype={"コード":"str"})
dfAll["資産価値を含めたPER"] = math.nan

dfAll = dfAll.sort_index()
dlCode = dfAll.groupby(level='コード').size().index.tolist()
#display(dlCode)
i=0
for code in dlCode:
    if (i>=0) & (i<=5000):
        print(code)
        try:
            balance_sheet = Ticker(code+".T").balance_sheet().reset_index().set_index("asOfDate")
        except AttributeError:
            balance_sheet = Ticker(code+".T").balance_sheet()
        #display(balance_sheet)
        if type(balance_sheet) != str:
            for date in balance_sheet.index:
                dtDate = datetime.date(date.year,date.month,1)
                strDate = datetime.date.strftime(dtDate, "%Y/%m")
                try:
                    CurrentAssets = balance_sheet.loc[date, "CurrentAssets"]
                except KeyError:
                    CurrentAssets = math.nan
                try:
                    NetDebt = balance_sheet.loc[date, "NetDebt"]
                except KeyError:
                    NetDebt = balance_sheet.loc[date, "TotalAssets"] - balance_sheet.loc[date, "StockholdersEquity"]
                try:
                    AvailableForSaleSecurities = balance_sheet.loc[date, "AvailableForSaleSecurities"]
                except KeyError:
                    AvailableForSaleSecurities = math.nan
                netCash = CurrentAssets + NetDebt * 0.7 -AvailableForSaleSecurities
                #print(netCash)
                #print(date)
                #display(dfAll.loc[(code, )].index)
                try:
                    dfAll.loc[(code, strDate), "資産価値を含めたPER"] = round(\
                    (dfAll.loc[(code, strDate), "時価総額"] - netCash) / dfAll.loc[(code, strDate), "純利益"], 2)
                except KeyError:
                    dfAll.loc[(code, strDate),:] = math.nan
                    dfAll.loc[(code, strDate), "資産価値を含めたPER"] = round(\
                    (dfAll.loc[(code, strDate), "時価総額"] - netCash) / dfAll.loc[(code, strDate), "純利益"], 2)
                #print(code, strDate, dfAll.loc[(code, strDate), "資産価値を含めたPER"])
    i+=1
    if i%10==0:
        print(i)
        dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv")
        #break
dfAll.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv")

## 最新日付のファイルの準備

In [ ]:
# 最新日付のみのデータフレームを作成する（※中時間）
import pandas as pd
import myLibStandard as myLib
myLib.set_DisplayOption()

dfAll = pd.read_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTable.csv",parse_dates=[1],index_col=["コード", "年度"], dtype={"コード":str})
dfAll = dfAll.sort_index()
dlCode = dfAll.groupby(level='コード').size().index.tolist()
dfMaxDate = pd.DataFrame()
for code in dlCode:
    maxDate = dfAll.loc[(code,)].index.max()
    #display(dfAll.loc[(code,maxDate)])
    #print(code)
    if maxDate.year >= 2024:
        ds = dfAll.loc[(code,maxDate)].to_frame().T
        dfMaxDate = pd.concat([dfMaxDate,ds],axis=0)
dfMaxDate.index.names = ["コード", "年度"]
dfMaxDate.to_csv(myLib.pathDataSummaryDomesticData + "DomesticDataTableMaxDate.csv")